## Импорт необходимого функционала

In [1]:
import os
import pickle
import numpy as np
import pandas as pd

## Загрузка данных

In [2]:
sub = pd.read_csv("../data/mars-private_test-reg.csv")
test = pd.read_csv("../data/mars-private_test-reg.csv").drop(['Фаза Hor', 'Фаза Ver'], axis=1)

## Загрузка моделей

In [3]:
models_path = os.listdir("../models/")

models = {}
for path in models_path:
    with open(f"../models/{path}", "rb") as file:
        models[path.split('.')[0]] = pickle.load(file)
        
del models_path

## Предсказания базовыми моделями и ансамблями

In [4]:
catboost_preds_t = models["cb"].predict(test)
lgbm_preds_t = models["lgbm"].predict(test)
xgb_preds_t = models["xgb"].predict(test)
rf_preds_t = models["rf"].predict(test)
et_preds_t = models["et"].predict(test)

breg_preds_t = models["best_bagging_model"].predict(test)
vot_preds_t = models["vot_reg"].predict(test)

bag_ext_preds_t = models["bag_ext"].predict(test)
bag_rf_preds_t = models["bag_rf"].predict(test)

rf_2_preds_t = models["rf2"].predict(test)
et_2_preds_t = models["et2"].predict(test)

In [5]:
X_meta_t = np.column_stack(
    (
        catboost_preds_t,
        lgbm_preds_t,
        xgb_preds_t,
        rf_preds_t,
        et_preds_t,
        breg_preds_t,
        vot_preds_t,
        bag_ext_preds_t,
        bag_rf_preds_t,
        rf_2_preds_t,
        et_2_preds_t
    )
)

## Предсказание метамоделью

In [6]:
meta_preds_t = models["meta_lr"].predict(X_meta_t)

## Оформление сабмита

In [7]:
sub["Доля сигнала в ВП"] = meta_preds_t

assert sub.shape[1] == 12

sub.to_csv("submission_reg.csv", index=False)

---